# Marketing Campaign Analysis: Identifying Drivers of Web Purchases  
**![](https://media.istockphoto.com/id/1253822466/vector/seo-analytics-optimization-desktop-dashboard-blue-flat-design.jpg?s=612x612&w=0&k=20&c=OznOihx_irQJY4baEM8wCirn80VS0oiSoTJ1lXcTZEs=)**

# Background of Analysis
Gourmet Delight is an online retailer that specializes in high-quality food and luxury items such as wine, fruit, meat products, fish products, and gold products. The company recently conducted a marketing campaign targeting 2,240 customers. The campaign data includes customer profiles, product preferences, campaign successes/failures, and channel performance and they would like to gain insights into their business operations and improve their marketing strategy by understanding the factors that influence web purchases, identifying the most successful marketing campaigns, understanding the average customer profile, identifying the best performing products, and identifying and addressing underperforming channels.

# Objective of the Analysis
You are tasked to provide answers to the following questions:
* Are there any null values or outliers? How will you handle them?

* What factors are significantly related to the number of web purchases?

* Which marketing campaign was the most successful?

* What does the average customer look like?

* Which products are performing best?

* Which channels are underperforming?

# Importing the Libraries

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import statsmodels.api as sm
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
%matplotlib inline

# Loading the Dataset

In [ ]:
marketing_df = pd.read_csv('/kaggle/input/marketing-dataset/marketing_data.csv')
marketing_df.head()

# About the Data


In [ ]:
# Load data dictionary file to better understand the data
marketing_dic_df = pd.read_csv('/kaggle/input/marketing-dataset/marketing_data_dictionary.csv')
marketing_dic_df.head(28)

# Exploring the data

In [ ]:
#check out the number of rows and column
print('Number of Rows and Columns ', marketing_df.shape)

In [ ]:
marketing_df.describe()

In [ ]:
marketing_df.info()

From the info above i noticed the following issues:
1. Income column has whitespace
2. Income column has 24 missing values
3. The column names are in two writing formats


## Working on the issues above:

In [ ]:
# issue 1 & 3
#removing whitespace and standadize the column name
marketing_df.columns = ['ID', 'Year_Birth', 'Education', 'Marital_Status', 'Income',
       'Kid_home', 'Teen_home', 'Dt_Customer', 'Recency', 'Mnt_Wines',
       'Mnt_Fruits', 'Mnt_Meat_Products', 'Mnt_Fish_Products', 'Mnt_Sweet_Products',
       'Mnt_Gold_Prods', 'Num_Deals_Purchases', 'Num_Web_Purchases',
       'Num_Catalog_Purchases', 'Num_Store_Purchases', 'Num_Web_Visits_Month',
       'AcceptedCmp3', 'AcceptedCmp4', 'AcceptedCmp5', 'AcceptedCmp1',
       'AcceptedCmp2', 'Response', 'Complain', 'Country']
# look at the columns
marketing_df.columns


# Question 1
Are there any null values or outliers? How will you handle them?

In [ ]:
# issue 2
#dropping the missing values in income since they are less than 1% of the datas
marketing_df.dropna()


In [ ]:
#check out the distribution of income
sns.histplot(data = marketing_df , x = 'Income')
plt.show()

from the histogram above income is between 0 to 200,000 and there seem to be an outlier which may be due to error during data entry, it seem to be just one so I'll be removing it

In [ ]:
#remove the outlier by removing values that are more than 200,000
marketing_df = marketing_df[marketing_df['Income'] <= 200000]
#view the distribution of income now
sns.boxplot(data = marketing_df, x = 'Income')
plt.show()

From this this distribution income can be grouped into 3 categories: low income, middle income and high income


# Feature Engineering
1. Creating Income level column

In [ ]:
#create a new column to group income into
def income_group_func(income):
    if income < 30000:
        return "low income"
    elif income >= 30000 and income < 70000:
        return "middle income"
    else:
        return "high income"

marketing_df["Income_Level"] = marketing_df["Income"].apply(income_group_func)
marketing_df["Income_Level"]


2. Creating Age column


In [ ]:
#create another factor 
#which is their ages in 2014 since that is the most recent enrollment date
marketing_df['Age'] = 2014 - marketing_df['Year_Birth'] 
#check out the distribution of the age
sns.histplot(data = marketing_df, x = 'Age')
plt.show()

3. Creating Age group column


In [ ]:
# group the age into age brackets
def age_group_func(age):
    if age < 30:
        return "young"
    elif age >= 30 and age < 60:
        return "middle age"
    else:
        return "old"

marketing_df["Age_Group"] = marketing_df["Age"].apply(age_group_func)
marketing_df["Age_Group"]

# Question 2
What factors are significantly related to the number of web purchases?

In [ ]:
#I'll be using regression model to identify variables that are significantly related to web purchases
#firstly, i'll convert categorical data into numerical variable
#using one-hot encoding to convert categerical data to binary variable
cat_vari = pd.get_dummies(marketing_df, columns = ['Education', 'Marital_Status', 'Country'])
#i'll drop variables that are not relevant
X = cat_vari.drop(['ID', 'Year_Birth', 'Dt_Customer','Response', 'Recency', 'Mnt_Wines',
       'Mnt_Fruits', 'Mnt_Meat_Products', 'Mnt_Fish_Products', 'Mnt_Sweet_Products',
       'Mnt_Gold_Prods', 'Num_Web_Purchases', 'Income_Level', 'Age_Group'], axis = 1) #independent variables
Y = marketing_df['Num_Web_Purchases'] #dependent variable
#split data
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, random_state = 42)
#create model
model = LinearRegression()
#fit model
model.fit(X_train, Y_train)
#make prediction
y_pred = model.predict(X_test)
# evaluate performance
mse = mean_squared_error(Y_test, y_pred)
r2 = r2_score(Y_test, y_pred)
print('MSE:', mse)
print('R-squared:', r2)

# view coefficients
coef_df = pd.DataFrame({'feature': X.columns, 'coef': model.coef_}).sort_values('coef', ascending=False)
print(coef_df)

# plot coefficients
import matplotlib.pyplot as plt
fig, ax = plt.subplots(figsize=(10,8))
ax.barh(coef_df['feature'], coef_df['coef'])
plt.show()


this shows the following:
* Customers who live alone tend to have higher web purchase
* Customers from Mexico also have higher web purchase
* Customers who have more kids at home tend to have lower web purchase
* Customer with marital status: Absurd and Widow have low web purchase
* Age and Income has almost not relationship with web purchase

# Question 3
Which marketing campaign was the most successful?

In [ ]:
#sum up the values in each campaign 
campaign_df = marketing_df[['AcceptedCmp1', 'AcceptedCmp2', 'AcceptedCmp3', 'AcceptedCmp4', 'AcceptedCmp5']]

campaign_df.sum().sort_values().plot(kind = 'bar')
plt.show()
campaign_df.sum().sort_values()

this shows that the 4th campaign is more successful, 5th and 3th campaign are close and also successful

# Question 4
What does the average customer look like?


In [ ]:
#calculate the average age, income, education and family size
avg_age = marketing_df['Age'].mean()
avg_income = marketing_df['Income'].mean()
avg_education = marketing_df['Education'].mode() #using mode to find the most common education
avg_marital_status = marketing_df['Marital_Status'].mode() 
avg_fam_size = marketing_df['Teen_home'].mean() + marketing_df['Kid_home'].mean()
print("The average age of customer is ", round(avg_age) )
print("The average income of customer is ", round(avg_income, 2) )
print("The average family size of customer is ", round(avg_fam_size) )
print("The average education of customer is ", avg_education )
print("The average marital status of customer is ", avg_marital_status )


The average customer is a 45 years old married graduate with 1 kid and a yearly household income of 51,969.86 


# Question 5
Which products are performing best?

In [ ]:
#sum up the values in each product
product_df = marketing_df[[ 'Mnt_Wines',
       'Mnt_Fruits', 'Mnt_Meat_Products', 'Mnt_Fish_Products', 'Mnt_Sweet_Products',
       'Mnt_Gold_Prods']]
product_df.sum().sort_values().plot(kind = 'bar')
plt.show()
product_df.sum().sort_values()

this shows that wine is the best performing product and next to it is meat

# Question 6
Which channels are underperforming?

In [ ]:
# #sum up the values in each product
channel_df = marketing_df[[ 'Num_Deals_Purchases', 'Num_Web_Purchases',
       'Num_Catalog_Purchases', 'Num_Store_Purchases']]
channel_df.sum().sort_values().plot(kind = 'bar')
plt.show()
channel_df.sum().sort_values()

These results indicate that the Deals channel has the lowest performance, with a total purchase of 5145.

# NB
* The senario stated above is a fictitious scenario and the company was created for the purpose of the analysis.
* The dataset was gotten from maven data playground
* All analysis were done by me and for my portfolio project